In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [13]:
raw_data = pd.read_csv('Example no. 1. (Exploratory, Classification) Classifying water potability.csv')

raw_data.head(5)

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.89,20791.32,7.30,368.52,564.31,10.38,86.99,2.96,0
1,3.716080,129.42,18630.06,6.64,NaN,592.89,15.18,56.33,4.50,0
2,8.099124,224.24,19909.54,9.28,NaN,418.61,16.87,66.42,3.06,0
3,8.316766,214.37,22018.42,8.06,356.89,363.27,18.44,100.34,4.63,0
4,9.092223,181.10,17978.99,6.55,310.14,398.41,11.56,32.00,4.08,0


In [14]:
raw_data.describe

<bound method NDFrame.describe of             ph  Hardness    Solids  Chloramines  Sulfate  Conductivity  \
0          NaN    204.89  20791.32         7.30   368.52        564.31   
1     3.716080    129.42  18630.06         6.64      NaN        592.89   
2     8.099124    224.24  19909.54         9.28      NaN        418.61   
3     8.316766    214.37  22018.42         8.06   356.89        363.27   
4     9.092223    181.10  17978.99         6.55   310.14        398.41   
...        ...       ...       ...          ...      ...           ...   
3271  4.668102    193.68  47580.99         7.17   359.95        526.42   
3272  7.808856    193.55  17329.80         8.06      NaN        392.45   
3273  9.419510    175.76  33155.58         7.35      NaN        432.04   
3274  5.126763    230.60  11983.87         6.30      NaN        402.88   
3275  7.874671    195.10  17404.18         7.51      NaN        327.46   

      Organic_carbon  Trihalomethanes  Turbidity  Potability  
0             

In [15]:
ph_na_df = raw_data[raw_data['ph'].isna()]
ph_na_df.head(5)

sulfate_na_df = raw_data[raw_data['Sulfate'].isna()]
sulfate_na_df.head(5)

trihalomethanes_na_df = raw_data[raw_data['Trihalomethanes'].isna()]
trihalomethanes_na_df.head(5)

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
62,NaN,229.49,35729.69,8.81,384.94,296.40,16.93,NaN,3.86,0
81,5.519126,168.73,12531.60,7.73,NaN,443.57,18.10,NaN,3.76,0
110,9.286155,222.66,12311.27,7.29,332.24,353.74,14.17,NaN,5.24,0
118,7.397413,122.54,8855.11,6.89,241.61,489.85,13.37,NaN,3.15,0
119,7.812804,196.58,42550.84,7.33,NaN,442.55,14.67,NaN,6.20,0


In [17]:
clean_data = raw_data.dropna()
clean_data.to_csv('clean_data.csv', index=False)
clean_data.head(5)
print("Total number of rows after cleaning: ", clean_data.shape[0])

print("Total number of rows before cleaning: ", raw_data.shape[0])



Total number of rows after cleaning:  2011
Total number of rows before cleaning:  3276


In [18]:
clean_data.describe()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
count,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000
mean,7.085990,195.968026,21917.441243,7.134291,333.224664,426.526474,14.357683,66.400855,3.969657,0.403282
std,1.573337,32.635028,8642.239802,1.584821,41.205226,80.712672,3.325022,16.077042,0.780294,0.490678
min,0.227499,73.490000,320.940000,1.390000,129.000000,201.620000,2.200000,8.580000,1.450000,0.000000
25%,6.089723,176.745000,15615.665000,6.140000,307.630000,366.680000,12.125000,55.955000,3.440000,0.000000
50%,7.027297,197.190000,20933.510000,7.140000,332.230000,423.460000,14.320000,66.540000,3.970000,0.000000
75%,8.052969,216.440000,27182.585000,8.110000,359.330000,482.370000,16.680000,77.295000,4.515000,1.000000
max,14.000000,317.340000,56488.670000,13.130000,481.030000,753.340000,27.010000,124.000000,6.490000,1.000000


In [30]:
# ------------------------------------------------------------
# Dự đoán khả năng uống được của nước (Potability) bằng Random Forest
# - Đọc clean_data.csv
# - Cân bằng dữ liệu bằng SMOTE
# - (Tuỳ chọn) Tối ưu tham số bằng RandomizedSearchCV
# - Huấn luyện, đánh giá, trực quan hoá
# ------------------------------------------------------------

# Lưu ý: Nếu chạy trong môi trường không cài sẵn imblearn (ví dụ Google Colab, Jupyter),
# hãy chạy: !pip install imbalanced-learn

import argparse
import json

from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
)

try:
    from imblearn.over_sampling import SMOTE
except ImportError:
    raise ImportError(
        "Thiếu thư viện imblearn. Hãy cài đặt bằng lệnh: pip install imbalanced-learn"
    )

import matplotlib.pyplot as plt

RANDOM_STATE = 42


def load_data(csv_path):
    # Chấp nhận cả str và Path
    csv_path = Path(csv_path)
    if not csv_path.exists():
        raise FileNotFoundError(f"Không tìm thấy file: {csv_path}")
    df = pd.read_csv(csv_path)
    required_cols = {
        "ph",
        "Hardness",
        "Solids",
        "Chloramines",
        "Sulfate",
        "Conductivity",
        "Organic_carbon",
        "Trihalomethanes",
        "Turbidity",
        "Potability",
    }
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Thiếu cột bắt buộc: {missing}")
    return df


def smote_split(df, test_size=0.3):
    X = df.drop(columns=["Potability"])
    y = df["Potability"].astype(int)

    # Cân bằng dữ liệu bằng SMOTE trên toàn bộ X, y trước khi chia
    smote = SMOTE(random_state=RANDOM_STATE)
    X_res, y_res = smote.fit_resample(X, y)

    X_train, X_test, y_train, y_test = train_test_split(
        X_res,
        y_res,
        test_size=test_size,
        random_state=RANDOM_STATE,
        stratify=y_res,
    )
    return X_train, X_test, y_train, y_test


def tune_hyperparams(X_train, y_train, n_iter=20):
    # Không đặt grid quá rộng để thời gian hợp lý
    param_dist = {
        "n_estimators": [100, 200, 300],
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
    }
    rf = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1)
    rs = RandomizedSearchCV(
        rf,
        param_distributions=param_dist,
        n_iter=n_iter,
        cv=3,
        scoring="accuracy",
        n_jobs=-1,
        random_state=RANDOM_STATE,
        verbose=1,
    )
    rs.fit(X_train, y_train)
    return rs.best_params_, rs.best_estimator_


def train_fixed(X_train, y_train):
    # Bộ tham số tốt & chạy nhanh (rút từ thử nghiệm trước)
    model = RandomForestClassifier(
        n_estimators=200,
        max_depth=30,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=RANDOM_STATE,
        n_jobs=-1,
    )
    model.fit(X_train, y_train)
    return model


def evaluate(model, X_test, y_test, outdir, feature_names):
    outdir = Path(outdir)
    outdir.mkdir(parents=True, exist_ok=True)

    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    report_dict = classification_report(y_test, y_pred, output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose()
    cm = confusion_matrix(y_test, y_pred)

    # Lưu báo cáo & confusion matrix số liệu
    report_path = outdir / "classification_report.csv"
    cm_path = outdir / "confusion_matrix.csv"
    report_df.to_csv(report_path, index=True, encoding="utf-8-sig")
    pd.DataFrame(cm, index=[0, 1], columns=[0, 1]).to_csv(
        cm_path, index=True, encoding="utf-8-sig"
    )

    # Vẽ Confusion Matrix (matplotlib thuần)
    plt.figure(figsize=(6, 5))
    plt.imshow(cm, interpolation="nearest")
    plt.title("Confusion Matrix - Random Forest (SMOTE)")
    plt.colorbar()
    tick_marks = np.arange(2)
    classes = ["Không uống (0)", "Uống được (1)"]
    plt.xticks(tick_marks, classes, rotation=15)
    plt.yticks(tick_marks, classes)
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(
                j,
                i,
                format(cm[i, j], "d"),
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black",
            )
    plt.ylabel("Thực tế")
    plt.xlabel("Dự đoán")
    plt.tight_layout()
    cm_fig = outdir / "confusion_matrix.png"
    plt.savefig(cm_fig, dpi=160)
    plt.close()

    # Feature importance
    importances = getattr(model, "feature_importances_", None)
    if importances is not None:
        fi = (
            pd.DataFrame({"Feature": feature_names, "Importance": importances})
            .sort_values("Importance", ascending=False)
            .reset_index(drop=True)
        )
        fi_path = outdir / "feature_importance.csv"
        fi.to_csv(fi_path, index=False, encoding="utf-8-sig")

        # Plot feature importance (matplotlib thuần)
        plt.figure(figsize=(10, 6))
        order = np.argsort(importances)[::-1]
        plt.barh(
            np.array(feature_names)[order][::-1],
            np.array(importances)[order][::-1],
        )
        plt.title("Tầm quan trọng của các đặc trưng - Random Forest (SMOTE)")
        plt.xlabel("Mức độ quan trọng")
        plt.tight_layout()
        fi_fig = outdir / "feature_importance.png"
        plt.savefig(fi_fig, dpi=160)
        plt.close()
    else:
        fi_path = None
        fi_fig = None

    # Lưu tóm tắt kết quả
    summary = {
        "accuracy": acc,
        "report_csv": str(report_path),
        "confusion_matrix_csv": str(cm_path),
        "confusion_matrix_png": str(cm_fig),
        "feature_importance_csv": str(fi_path) if importances is not None else None,
        "feature_importance_png": str(fi_fig) if importances is not None else None,
        "best_params": getattr(model, "get_params", lambda: {})(),
    }
    with open(outdir / "summary.json", "w", encoding="utf-8") as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)

    return acc, report_df, cm, summary


def main():
    parser = argparse.ArgumentParser(
        description="Dự đoán Potability bằng Random Forest + SMOTE"
    )
    parser.add_argument(
        "--csv",
        type=str,
        default="clean_data.csv",
        help="Đường dẫn tới clean_data.csv",
    )
    parser.add_argument(
        "--outdir",
        type=str,
        default="outputs_rf_smote",
        help="Thư mục lưu kết quả",
    )
    parser.add_argument(
        "--tune",
        action="store_true",
        help="Bật RandomizedSearchCV để tối ưu tham số",
    )
    parser.add_argument(
        "--n_iter",
        type=int,
        default=20,
        help="Số lần lặp RandomizedSearchCV (khi bật --tune)",
    )
    # Nếu chạy trong notebook, bỏ qua parse_args()
    import sys
    if hasattr(sys, "argv") and (sys.argv[0].endswith(".ipynb") or "ipykernel" in sys.modules):
        # Chạy trong notebook: dùng giá trị mặc định
        class Args:
            csv = "clean_data.csv"
            outdir = "outputs_rf_smote"
            tune = False
            n_iter = 20
        args = Args()
    else:
        args = parser.parse_args()

    csv_path = args.csv
    outdir = args.outdir

    print("➡️ Đang nạp dữ liệu…")
    df = load_data(csv_path)
    feature_names = [c for c in df.columns if c != "Potability"]

    print("➡️ Đang SMOTE + chia train/test…")
    X_train, X_test, y_train, y_test = smote_split(df)

    if getattr(args, "tune", False):
        print("➡️ RandomizedSearchCV đang chạy (có thể mất vài phút)…")
        best_params, model = tune_hyperparams(X_train, y_train, n_iter=getattr(args, "n_iter", 20))
        print("✅ Best params:", best_params)
    else:
        print("➡️ Dùng bộ tham số nhanh (đã tối ưu trước).")
        model = train_fixed(X_train, y_train)

    print("➡️ Đang đánh giá mô hình…")
    acc, report_df, cm, summary = evaluate(
        model, X_test, y_test, outdir, feature_names
    )

    print("\n===== KẾT QUẢ =====")
    print(f"Accuracy: {acc:.4f}")
    print("\nClassification report:")
    print(report_df.to_string())
    print("\nConfusion matrix:")
    print(pd.DataFrame(cm, index=[0, 1], columns=[0, 1]).to_string())
    print(f"\n📁 Đã lưu kết quả vào thư mục: {Path(outdir).resolve()}")
    print("(summary.json chứa đường dẫn đầy đủ tới các file kết quả.)")


if __name__ == "__main__":
    # Nếu chạy trong notebook, không gọi main() với parse_args()
    import sys
    if hasattr(sys, "argv") and (sys.argv[0].endswith(".ipynb") or "ipykernel" in sys.modules):
        main()
    else:
        main()


➡️ Đang nạp dữ liệu…
➡️ Đang SMOTE + chia train/test…
➡️ Dùng bộ tham số nhanh (đã tối ưu trước).
➡️ Đang đánh giá mô hình…

===== KẾT QUẢ =====
Accuracy: 0.6986

Classification report:
              precision    recall  f1-score     support
0              0.698061  0.700000  0.699029  360.000000
1              0.699164  0.697222  0.698192  360.000000
accuracy       0.698611  0.698611  0.698611    0.698611
macro avg      0.698613  0.698611  0.698611  720.000000
weighted avg   0.698613  0.698611  0.698611  720.000000

Confusion matrix:
     0    1
0  252  108
1  109  251

📁 Đã lưu kết quả vào thư mục: D:\99_MasterDegree\02_Course\UNG DUNG CONG NGHE MOI TRONG CTN\Repo\HCMUNRE\Practise\outputs_rf_smote
(summary.json chứa đường dẫn đầy đủ tới các file kết quả.)


In [33]:
# Thêm cột dự đoán (0/1) vào dữ liệu và xuất file mới
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from pathlib import Path
import os

# Đọc dữ liệu gốc
# Sửa lỗi: kiểm tra đường dẫn, nếu không tồn tại thì báo lỗi rõ ràng
in_path = Path("clean_data.csv")
if not in_path.exists():
    # Thử tìm trong thư mục hiện tại
    alt_path = Path.cwd() / "clean_data.csv"
    if alt_path.exists():
        in_path = alt_path
    else:
        raise FileNotFoundError(
            f"Không tìm thấy file dữ liệu: {in_path.resolve()}\n"
            "Hãy chắc chắn rằng file 'clean_data.csv' nằm trong thư mục làm việc hiện tại."
        )

df = pd.read_csv(in_path)

# Kiểm tra cột Potability có tồn tại không
if "Potability" not in df.columns:
    raise ValueError("File dữ liệu không có cột 'Potability'.")

# Chuẩn bị X, y
X = df.drop(columns=["Potability"])
y = df["Potability"].astype(int)

# Cân bằng dữ liệu để huấn luyện
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Huấn luyện mô hình (bộ tham số đã dùng trước đó, chạy nhanh & cho chất lượng tốt)
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=30,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_res, y_res)

# Dự đoán cho TOÀN BỘ dữ liệu gốc
preds = rf.predict(X).astype(int)

# Thêm cột dự đoán
df["Predicted_Potability"] = preds  # 0 = không uống được, 1 = uống được

# Lưu file mới
out_path = Path("clean_data_with_predictions.csv")
df.to_csv(out_path, index=False, encoding="utf-8-sig")

# Hiển thị bảng để bạn xem trực tiếp (nếu dùng notebook, dùng display)
try:
    from IPython.display import display
    display(df)
except ImportError:
    print(df.head())

# In tóm tắt phân bố nhãn dự đoán
pred_counts = df["Predicted_Potability"].value_counts().sort_index()
print("Tổng quan dự đoán:")
print(f"0 (không uống được): {pred_counts.get(0, 0)} mẫu")
print(f"1 (uống được): {pred_counts.get(1, 0)} mẫu")
print(f"Đã lưu file kết quả: {out_path.resolve()}")


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability,Predicted_Potability
0,8.316766,214.37,22018.42,8.06,356.89,363.27,18.44,100.34,4.63,0,0
1,9.092223,181.10,17978.99,6.55,310.14,398.41,11.56,32.00,4.08,0,0
2,5.584087,188.31,28748.69,7.54,326.68,280.47,8.40,54.92,2.56,0,0
3,10.223862,248.07,28749.72,7.51,393.66,283.65,13.79,84.60,2.67,0,0
4,8.635849,203.36,13672.09,4.56,303.31,474.61,12.36,62.80,4.40,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2006,8.989900,215.05,15921.41,6.30,312.93,390.41,9.90,55.07,4.61,1,1
2007,6.702547,207.32,17246.92,7.71,304.51,329.27,16.22,28.88,3.44,1,1
2008,11.491011,94.81,37188.83,9.26,258.93,439.89,16.17,41.56,4.37,1,1
2009,6.069616,186.66,26138.78,7.75,345.70,415.89,12.07,60.42,3.67,1,1


Tổng quan dự đoán:
0 (không uống được): 1200 mẫu
1 (uống được): 811 mẫu
Đã lưu file kết quả: D:\99_MasterDegree\02_Course\UNG DUNG CONG NGHE MOI TRONG CTN\Repo\HCMUNRE\Practise\clean_data_with_predictions.csv
